In [1]:
import csv
import pandas as pd
import numpy as np

In [2]:
with open("MW.txt") as data:
    reader = csv.reader(data, delimiter='\t')
    d = list(reader)

for i in d:
    i[6]=[int(i) for i in  i[6].split(',')]
    i[6]=i[6][0:441]

In [3]:
j=1000
for m in d:
    if int(m[5])<j:
        j=int(m[5])
print(j)
print(d[1])

441
['1', '1', 'MW', 'FP1', '1', '889', [83, 74, 65, 65, 66, 55, 43, 25, 18, 20, 26, 33, 41, 38, 28, 36, 55, 60, 56, 58, 72, 85, 76, 69, 74, 83, 86, 88, 97, 112, 114, 109, 99, 76, 54, 50, 66, 86, 97, 99, 96, 86, 82, 73, 69, 70, 70, 60, 65, 71, 69, 64, 57, 54, 56, 58, 50, 20, -20, -56, -74, -69, -57, -57, -60, -69, -75, -71, -59, -45, -24, 3, 13, 10, 8, 10, 5, 7, 8, -3, -22, -28, -20, 1, 3, -10, -18, -12, -19, -23, -25, -36, -45, -43, -37, -23, -4, 4, -2, -3, -3, -2, 5, 8, 7, 5, 16, 29, 34, 35, 36, 48, 68, 80, 80, 75, 68, 69, 77, 89, 105, 100, 80, 64, 56, 59, 64, 57, 49, 42, 48, 60, 75, 77, 70, 61, 45, 28, 24, 28, 29, 34, 40, 50, 42, 27, 16, 4, 1, 6, 12, 16, 20, 24, 34, 44, 54, 58, 53, 40, 24, 11, 4, -4, -14, -34, -56, -76, -77, -68, -53, -44, -38, -35, -30, -30, -30, -25, -6, 18, 29, 27, 23, 27, 41, 50, 51, 57, 57, 51, 55, 57, 58, 53, 37, 20, 9, 12, 29, 49, 49, 37, 28, 37, 57, 76, 83, 74, 68, 50, 25, 9, 13, 39, 71, 103, 121, 130, 132, 136, 146, 163, 184, 188, 171, 152, 152, 164, 166, 1

In [4]:
dataset = pd.DataFrame(d)
X = dataset.iloc[:, 6].values
y = dataset.iloc[:, 4].values
x=[]

for i in X:
    x.append(i)
X = pd.DataFrame(x)
print(y.shape)
X.shape

(67635,)


(67635, 441)

In [5]:
X['Class']=y
X['Class'].value_counts()
X[X["Class"]=='1'].count()

0        5498
1        5498
2        5498
3        5498
4        5498
         ... 
437      5498
438      5498
439      5498
440      5498
Class    5498
Length: 442, dtype: int64

In [6]:
X.isnull().sum()

0        0
1        0
2        0
3        0
4        0
        ..
437      0
438      0
439      0
440      0
Class    0
Length: 442, dtype: int64

In [7]:
c_mean = pd.DataFrame()
for c, f_vec in X.groupby('Class'):
    c_mean[c] = f_vec.mean()

In [8]:
Sw = np.zeros((441,441))
for c, f_vec in X.groupby('Class'):
    f_vec = f_vec.drop(['Class'], axis=1)          
    si = np.zeros((441,441))
    for index, row in f_vec.iterrows():
        x= row.values.reshape([441,1])
        mi = c_mean[c].values.reshape([441,1])
        si += np.matmul((x - mi),(x - mi).T)
    Sw += si                                  

In [9]:
T_mean = X.mean()  
X['Class'].value_counts()
Sb = np.zeros((441,441))
for c in c_mean:
    n = X[X["Class"]==str(c)].count()["Class"]
    mi= c_mean[c].values.reshape(441,1)
    m = T_mean.values.reshape(441,1)
    Sb += n * np.matmul((mi - m),(mi - m).T)

St =Sw + Sb
X = X.drop(labels='Class', axis=1)

In [10]:
eig_val, eig_vec = np.linalg.eig(np.matmul(np.linalg.inv(Sw),Sb))
eig_p = [(np.abs(eig_val[i]), eig_vec[:,i]) for i in range(len(eig_val))]
eig_p = sorted(eig_p, key=lambda x: x[0], reverse=True)
W=np.vstack((eig_p[0][1],eig_p[1][1],eig_p[2][1],eig_p[3][1],eig_p[4][1],eig_p[5][1],eig_p[6][1],eig_p[7][1],eig_p[8][1],eig_p[9][1])).real
Xp = np.array(np.dot(X,W.T))
print(Xp.shape)
#total number of classes = 11

(67635, 10)


In [11]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC 
from sklearn import preprocessing
Xp = preprocessing.scale(Xp)
X_train, X_test, y_train, y_test = train_test_split(Xp, y, test_size = 0.20)

In [12]:
svclassifier = SVC(kernel='rbf')
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)
score=svclassifier.score(X_test,y_test)
print(score)



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.24920529311746878


In [13]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


[[2123   48   72   36   24   54   30   23   30   40   23]
 [ 241  151  124   69   82   90   74   91   72   73   65]
 [ 259   87  237   71   84   80   85   63   59   66   46]
 [ 230   98  120  102   75   99   61   88   67   62   67]
 [ 196   99  146   61  106   68   71   70   67   87   53]
 [ 235  107   88   76   73  145   50   63   89   78   67]
 [ 231  106  141   83  100  100  117   79   80   86   60]
 [ 206   95  118   67   73  107   72   88   72   72   54]
 [ 207  104  123   84   89  124   58   94  105   76   69]
 [ 248  105  142   70   75  104   73   72   50  108   73]
 [ 214  115  134   67   83  116   74   73   82   84   89]]
              precision    recall  f1-score   support

          -1       0.48      0.85      0.62      2503
           0       0.14      0.13      0.13      1132
           1       0.16      0.21      0.18      1137
           2       0.13      0.10      0.11      1069
           3       0.12      0.10      0.11      1024
           4       0.13      0.14   